In [ ]:
#the module 'sys' allows istalling module from inside Jupyter
import sys

!{sys.executable} -m pip install numpy
import numpy as np

!{sys.executable} -m pip install pandas
import pandas as pd

#Natrual Language ToolKit (NLTK)
!{sys.executable} -m pip install nltk
import nltk

!{sys.executable} -m pip install sklearn
from sklearn import metrics
#from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import  CountVectorizer #bag-of-words vectorizer
from sklearn.decomposition import LatentDirichletAllocation #package for LDA

# Plotting tools

from pprint import pprint
!{sys.executable} -m pip install pyLDAvis #visualizing LDA
import pyLDAvis
import pyLDAvis.lda_model

import matplotlib.pyplot as plt
%matplotlib inline

#define text normalization function
%run ./Text_Normalization_Function.ipynb #defining text normalization function

#ignore warnings about future changes in functions as they take too much space
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [57]:
import pandas as pd
df = pd.read_csv("train.csv")
df.head()


,Unnamed: 0,post_date,tweet,ticker_symbol
0,2,2015-08-26,$AMZN Reversal that quick from B&R,AMZN
1,3,2019-09-10,Great day in options $TSLA $BA $AAPL $ROKU $NV...,AAPL
2,4,2019-05-16,$AAPL continues to sell off premarket on no news.,AAPL
3,5,2018-01-24,Whats it like being on the opposite trade of @...,TSLA
4,6,2017-08-04,You are wrong. The reason $tsla stock performe...,TSLA


In [58]:
df_appl = df.loc[df.ticker_symbol == "AAPL"]
df_amzn = df.loc[df.ticker_symbol == "AMZN"]
df_goog = df.loc[df.ticker_symbol == "GOOG"]
df_msft = df.loc[df.ticker_symbol == "MSFT"]
df_tsla = df.loc[df.ticker_symbol == "TSLA"]
df_appl.shape

(21281, 4)

In [59]:
appl_corpus = list(df_appl["tweet"])
normalized_appl_corpus = normalize_corpus(appl_corpus)


In [60]:
#define the bag-of-words vectorizer:
bow_vectorizer = CountVectorizer(max_features=1000)

#vectorize the normalized data:
bow_appl_corpus = bow_vectorizer.fit_transform(normalized_appl_corpus)

pd.DataFrame(data = bow_appl_corpus.todense(), columns = bow_vectorizer.get_feature_names_out())

,1st,2nd,5g,aal,aapl,aapls,able,absolutely,acb,account,...,year,yes,yesterday,yet,yhoo,ym,yoy,yr,ytd,zone
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21276,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21277,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21278,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21279,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
lda_appl = LatentDirichletAllocation(n_components=3, max_iter=10,
                                           doc_topic_prior = 0.9,
                                           topic_word_prior = 0.9).fit(bow_appl_corpus)

In [62]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

def get_topic_words(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_words = []
    for topic_weights in lda_model.components_:
        top_word_locs = (-topic_weights).argsort()[:n_words]
        topic_words.append(keywords.take(top_word_locs).tolist())
    return topic_words

In [63]:
no_top_words = 15
display_topics(lda_appl, bow_vectorizer.get_feature_names_out(), no_top_words)

Topic 0:
aapl apple buy stock call look sell today earnings long take year share big put
Topic 1:
aapl amzn fb spy nflx tsla qqq goog msft googl stock twtr spx nvda baba
Topic 2:
aapl apple market good low day new trade high iphone like watch price bad short


In [64]:
word_weights = lda_appl.components_ / lda_appl.components_.sum(axis=1)[:, np.newaxis]
pd.DataFrame(word_weights.T, index = bow_vectorizer.get_feature_names_out()).T

,1st,2nd,5g,aal,aapl,aapls,able,absolutely,acb,account,...,year,yes,yesterday,yet,yhoo,ym,yoy,yr,ytd,zone
0,0.000683,0.000687,0.000757,0.000026,0.116928,0.001292,0.000047,0.000102,0.000014,0.000796,...,0.009469,0.000149,0.001858,0.000955,0.000018,0.000016,0.000617,0.001304,0.000268,0.000046
1,0.000063,0.000038,0.000019,0.000723,0.059844,0.000021,0.000027,0.000047,0.001503,0.000171,...,0.000061,0.000026,0.000044,0.000033,0.001326,0.001583,0.000016,0.000023,0.000392,0.000032
2,0.000064,0.000066,0.000024,0.000017,0.138488,0.000455,0.000920,0.000546,0.000016,0.000025,...,0.000162,0.001766,0.001433,0.001951,0.000018,0.000018,0.000016,0.000028,0.000026,0.000745


In [65]:
word_weights = lda_appl.components_ / lda_appl.components_.sum(axis=1)[:, np.newaxis]
word_weights_df = pd.DataFrame(word_weights.T,
                               index = bow_vectorizer.get_feature_names_out(),
                               columns = ["Topic_" + str(i) for i in range(3)])
word_weights_df.head(10)

,Topic_0,Topic_1,Topic_2
1st,0.000683,0.000063,0.000064
2nd,0.000687,0.000038,0.000066
5g,0.000757,0.000019,0.000024
aal,0.000026,0.000723,0.000017
aapl,0.116928,0.059844,0.138488
aapls,0.001292,0.000021,0.000455
able,0.000047,0.000027,0.000920
absolutely,0.000102,0.000047,0.000546
acb,0.000014,0.001503,0.000016
account,0.000796,0.000171,0.000025


In [66]:
word_weights_df.sort_values(by='Topic_0',ascending=False).head(10)

,Topic_0,Topic_1,Topic_2
aapl,0.116928,0.059844,0.138488
apple,0.021431,0.000074,0.026185
buy,0.018631,0.002580,0.003199
stock,0.015587,0.018984,0.004172
call,0.012623,0.000424,0.000084
look,0.011937,0.000283,0.001962
sell,0.011281,0.004065,0.001531
today,0.010850,0.005451,0.004691
earnings,0.010354,0.000079,0.000824
long,0.009706,0.000037,0.000345


In [67]:
lda_appl_topic_weights = lda_appl.transform(bow_appl_corpus )

In [68]:
#array of document "names" and topic "names" ("names" are just indecies)
doc_names = ["Doc_" + str(i) for i in range(len(normalized_appl_corpus))]
topic_names = ["Topic_" + str(i) for i in range(3)]

#convert to dataframe
df_document_topic1 = pd.DataFrame(np.round(lda_appl_topic_weights, 4), columns=topic_names, index=doc_names)
df_document_topic1.head(10)

,Topic_0,Topic_1,Topic_2
Doc_0,0.3413,0.5038,0.1550
Doc_1,0.2358,0.1859,0.5782
Doc_2,0.5760,0.2994,0.1246
Doc_3,0.4308,0.0808,0.4883
Doc_4,0.2492,0.2115,0.5392
Doc_5,0.5624,0.1828,0.2549
Doc_6,0.5132,0.0906,0.3962
Doc_7,0.3637,0.1845,0.4518
Doc_8,0.6229,0.2601,0.1170
Doc_9,0.5665,0.2795,0.1540


In [69]:
#vector of indecies for columns with the highest value by each row in df_document_topic
dominant_topic1 = np.argmax(df_document_topic1.values, axis=1)

#add dominant_topic as a column to df_document_topic
df_document_topic1['dominant_topic'] = dominant_topic1
df_document_topic1.head()

,Topic_0,Topic_1,Topic_2,dominant_topic
Doc_0,0.3413,0.5038,0.1550,1
Doc_1,0.2358,0.1859,0.5782,2
Doc_2,0.5760,0.2994,0.1246,0
Doc_3,0.4308,0.0808,0.4883,2
Doc_4,0.2492,0.2115,0.5392,2


In [70]:
stop = 0
for text in normalized_appl_corpus:
  print(text)
  print("-----------------------------------------")
  stop +=1
  if stop == 10:
    break

#normalized_appl_corpus

great day option tsla ba aapl roku nvda nflx incredibly solid p l oh man comment part spreadsheet dedicate spy lol typical
-----------------------------------------
aapl continue sell premarket news
-----------------------------------------
uber self driving car kill pedestrian maker self driving autonomous car liable accident auto maker want take liability customer auto insurer gm f tsla tm car aapl goog
-----------------------------------------
tim cook might good time use open market buyback actually defend b trade war nonsense option idiot load crash aapl wartime cfo take offense
-----------------------------------------
aapl pay thousand vending machine parking meter
-----------------------------------------
right aapl hedge gld even btc great hope stay aapl less short equity love yr return
-----------------------------------------
aapl approach dma first time eon clear level may well pop
-----------------------------------------
aapl yhoo call rip offs people buy 110s tuesday pay

In [71]:
# 0: negative; 1: neutral; 2: Positive
# replace them
df_document_topic1.loc[df_document_topic1['dominant_topic'] == 0, "dominant_topic"] = "negative"
df_document_topic1.loc[df_document_topic1["dominant_topic"] == 1, "dominant_topic"] = "neutral"
df_document_topic1.loc[df_document_topic1["dominant_topic"]== 2, "dominant_topic"] = "positive"
df_document_topic1.head()
df_document_topic1.shape



(21281, 4)

In [72]:
#df.info()
date_series = df.loc[df["ticker_symbol"]=="AAPL","post_date"]

df_temp = date_series.to_frame()
#df_temp

new_index = ['Doc_{}'.format(i) for i in range(len(date_series))]

# Assign this new index to your DataFrame
df_temp.index = new_index
df_temp

df_document_topic1['post_date'] = df_temp
df_document_topic1.head()

,Topic_0,Topic_1,Topic_2,dominant_topic,post_date
Doc_0,0.3413,0.5038,0.1550,neutral,2019-09-10
Doc_1,0.2358,0.1859,0.5782,positive,2019-05-16
Doc_2,0.5760,0.2994,0.1246,negative,2018-03-20
Doc_3,0.4308,0.0808,0.4883,positive,2018-04-04
Doc_4,0.2492,0.2115,0.5392,positive,2015-01-27


In [73]:
df_document_topic1.sort_values(by = "post_date")

,Topic_0,Topic_1,Topic_2,dominant_topic,post_date
Doc_5362,0.5636,0.1279,0.3085,negative,2015-01-01
Doc_19631,0.4987,0.0807,0.4207,negative,2015-01-01
Doc_13298,0.2421,0.1424,0.6154,positive,2015-01-01
Doc_17315,0.1633,0.7570,0.0797,neutral,2015-01-01
Doc_13826,0.2396,0.1151,0.6453,positive,2015-01-02
...,...,...,...,...,...
Doc_17568,0.4805,0.4180,0.1016,negative,2019-12-31
Doc_19362,0.7117,0.1684,0.1199,negative,2019-12-31
Doc_3532,0.2711,0.6350,0.0939,neutral,2019-12-31
Doc_18683,0.5738,0.0588,0.3674,negative,2019-12-31


In [74]:
# calculate the ups and downs of the stock prices
appl_stock_price = pd.read_csv("appl_stock_price.csv")
appl_stock_price ['Close_diff'] = appl_stock_price ['Close'].diff()
appl_stock_price['Date'] = pd.to_datetime(appl_stock_price['Date']).dt.date
appl_stock_price.head()

,Date,Close,Close_diff
0,2015-01-02,27.33,NaN
1,2015-01-05,26.56,-0.77
2,2015-01-06,26.57,0.01
3,2015-01-07,26.94,0.37
4,2015-01-08,27.97,1.03


In [75]:
# Convert 'post_date' to datetime if it's not already
df_document_topic1['post_date'] = pd.to_datetime(df_document_topic1['post_date'])

# Sort the DataFrame by 'post_date'
df_document_topic1 = df_document_topic1.sort_values('post_date')

# Initialize a list to store results
results = []

# Start from the first desired start date
current_date = pd.to_datetime('2015-01-01')

# Define the end date for the loop
loop_end_date = pd.to_datetime('2019-12-31')

while current_date <= loop_end_date:
    # Define the date range
    start_date = current_date
    end_date = start_date + pd.Timedelta(days=4)

    # Filter the DataFrame for the date range
    mask = (df_document_topic1['post_date'] >= start_date) & (df_document_topic1['post_date'] <= end_date)
    filtered_df = df_document_topic1.loc[mask]

    # Calculate the counts for 'dominant_topic'
    counts = filtered_df['dominant_topic'].value_counts()

    # Append the results, including the date range as a string
    results.append({
        'start_date': start_date.strftime('%Y-%m-%d'),
        'end_date': end_date.strftime('%Y-%m-%d'),
        'positive': counts.get('positive', 0),
        'neutral': counts.get('neutral', 0),
        'negative': counts.get('negative', 0)
    })

    # Move to the next five-day period
    current_date = start_date + pd.Timedelta(days=1)

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
results_df

,start_date,end_date,positive,neutral,negative
0,2015-01-01,2015-01-05,11,5,14
1,2015-01-02,2015-01-06,18,7,20
2,2015-01-03,2015-01-07,18,5,19
3,2015-01-04,2015-01-08,25,6,35
4,2015-01-05,2015-01-09,32,9,45
...,...,...,...,...,...
1821,2019-12-27,2019-12-31,18,19,21
1822,2019-12-28,2020-01-01,12,17,15
1823,2019-12-29,2020-01-02,12,14,13
1824,2019-12-30,2020-01-03,12,10,13


In [80]:
results_df.shape
results_df.head()

,start_date,end_date,positive,neutral,negative,aligned_date
0,2015-01-01,2015-01-05,11,5,14,2015-01-04
1,2015-01-02,2015-01-06,18,7,20,2015-01-05
2,2015-01-03,2015-01-07,18,5,19,2015-01-06
3,2015-01-04,2015-01-08,25,6,35,2015-01-07
4,2015-01-05,2015-01-09,32,9,45,2015-01-08


In [81]:
appl_stock_price.head()

,Date,Close,Close_diff
0,2015-01-02,27.33,NaN
1,2015-01-05,26.56,-0.77
2,2015-01-06,26.57,0.01
3,2015-01-07,26.94,0.37
4,2015-01-08,27.97,1.03


In [87]:
results_df['end_date'] = pd.to_datetime(results_df['end_date'])
# Create a new column in df2 that is 'end_date' minus one day
results_df['aligned_date'] = results_df['end_date'] + pd.Timedelta(days=1)

appl_stock_price['Date'] = pd.to_datetime(appl_stock_price['Date'])
results_df['aligned_date'] = pd.to_datetime(results_df['aligned_date'])
# Now merge df1 with df2 on 'Date' and 'aligned_date'
# We're using a left join here to keep all rows from the first table
combined_df = pd.merge(results_df, appl_stock_price, left_on='aligned_date', right_on='Date', how='left')

In [88]:
combined_df.head()

,start_date,end_date,positive,neutral,negative,aligned_date,Date,Close,Close_diff
0,2015-01-01,2015-01-05,11,5,14,2015-01-06,2015-01-06,26.57,0.01
1,2015-01-02,2015-01-06,18,7,20,2015-01-07,2015-01-07,26.94,0.37
2,2015-01-03,2015-01-07,18,5,19,2015-01-08,2015-01-08,27.97,1.03
3,2015-01-04,2015-01-08,25,6,35,2015-01-09,2015-01-09,28.00,0.03
4,2015-01-05,2015-01-09,32,9,45,2015-01-10,NaT,NaN,NaN


In [124]:
clean_combined_df = combined_df.drop(["aligned_date", "Close"], axis=1)

# eliminate the NaN
clean_combined_df_final = clean_combined_df.loc[~clean_combined_df["Close_diff"].isnull(),]
clean_combined_df_final

#clean_combined_df_final.info()

,start_date,end_date,positive,neutral,negative,Date,Close_diff
0,2015-01-01,2015-01-05,11,5,14,2015-01-06,0.01
1,2015-01-02,2015-01-06,18,7,20,2015-01-07,0.37
2,2015-01-03,2015-01-07,18,5,19,2015-01-08,1.03
3,2015-01-04,2015-01-08,25,6,35,2015-01-09,0.03
6,2015-01-07,2015-01-11,22,6,31,2015-01-12,-0.69
...,...,...,...,...,...,...,...
1813,2019-12-19,2019-12-23,19,17,23,2019-12-24,0.07
1815,2019-12-21,2019-12-25,16,10,19,2019-12-26,1.41
1816,2019-12-22,2019-12-26,20,11,25,2019-12-27,-0.03
1819,2019-12-25,2019-12-29,13,14,19,2019-12-30,0.43


In [111]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X = clean_combined_df_final.drop(['Close_diff','start_date', 'end_date', 'Date'], axis=1)
y = clean_combined_df_final['Close_diff']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

#print(predictions)
print(model.coef_)
print(model.intercept_)


[-0.00593298 -0.00318463  0.00660937]
0.057449558035888984


In [125]:
clean_combined_df_final_log = clean_combined_df_final
clean_combined_df_final_log['Close_diff'] = np.where(clean_combined_df_final_log["Close_diff"] > 0, "Up", "Down")
print(clean_combined_df_final_log)

      start_date   end_date  positive  neutral  negative       Date Close_diff
0     2015-01-01 2015-01-05        11        5        14 2015-01-06         Up
1     2015-01-02 2015-01-06        18        7        20 2015-01-07         Up
2     2015-01-03 2015-01-07        18        5        19 2015-01-08         Up
3     2015-01-04 2015-01-08        25        6        35 2015-01-09         Up
6     2015-01-07 2015-01-11        22        6        31 2015-01-12       Down
...          ...        ...       ...      ...       ...        ...        ...
1813  2019-12-19 2019-12-23        19       17        23 2019-12-24         Up
1815  2019-12-21 2019-12-25        16       10        19 2019-12-26         Up
1816  2019-12-22 2019-12-26        20       11        25 2019-12-27       Down
1819  2019-12-25 2019-12-29        13       14        19 2019-12-30         Up
1820  2019-12-26 2019-12-30        18       20        22 2019-12-31         Up

[1256 rows x 7 columns]


<ipython-input-125-8439e5460235>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_combined_df_final_log['Close_diff'] = np.where(clean_combined_df_final_log["Close_diff"] > 0, "Up", "Down")


In [137]:
from sklearn.linear_model import LogisticRegression

X = clean_combined_df_final.drop(['Close_diff','start_date', 'end_date', 'Date'], axis=1)
y = clean_combined_df_final['Close_diff']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
probabilities = model.predict_proba(X_test)

print(predictions)

# Coefficients and intercept
print(model.coef_)
print(model.intercept_)


['Up' 'Up' 'Down' 'Up' 'Up' 'Down' 'Down' 'Up' 'Up' 'Up' 'Up' 'Up' 'Down'
 'Up' 'Down' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Down' 'Down' 'Up' 'Up' 'Up'
 'Up' 'Up' 'Up' 'Down' 'Up' 'Up' 'Up' 'Down' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up'
 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Down' 'Up' 'Up' 'Down' 'Up' 'Up'
 'Up' 'Up' 'Up' 'Down' 'Up' 'Up' 'Up' 'Down' 'Down' 'Up' 'Up' 'Up' 'Up'
 'Up' 'Up' 'Down' 'Up' 'Down' 'Down' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up'
 'Up' 'Down' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up'
 'Down' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Down' 'Up' 'Up' 'Up' 'Up' 'Up'
 'Up' 'Up' 'Down' 'Up' 'Up' 'Up' 'Down' 'Up' 'Down' 'Up' 'Up' 'Up' 'Up'
 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Down' 'Up' 'Down'
 'Up' 'Up' 'Down' 'Down' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up'
 'Up' 'Up' 'Up' 'Up' 'Up' 'Down' 'Down' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up'
 'Up' 'Up' 'Up' 'Up' 'Down' 'Up' 'Up' 'Up' 'Up' 'Down' 'Up' 'Up' 'Up' 'Up'
 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up' 'Up'

In [127]:
from sklearn import metrics
print('Accuracy Rate:', metrics.accuracy_score(y_test, predictions),"\n")

Accuracy Rate: 0.5396825396825397 



In [140]:
# number of corrected prediction: 136
# total number of prediction: 252


total =0
for i in range(len(list(y_test))):
  if list(y_test)[i] == predictions[i]:
    total +=1
print("number of corrected prediction: " + str(total))
print("total number of prediction: " + str(len(predictions)))

number of corrected prediction: 136
total number of prediction: 252


In [135]:
pd.crosstab(pd.Series(list(y_test)),
            pd.Series(predictions),
            rownames = ['True:'],
            colnames = ['Predicted:'],
            margins = True)

Predicted:,Down,Up,All
True:,,,
Down,15,96,111
Up,20,121,141
All,35,217,252
